In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [5]:
dataset = keras.preprocessing.image_dataset_from_directory('dataset/', batch_size=64)

for  data, labels in dataset:
    print(data.shape)
    print(data.dtype)
    print(labels.shape)
    print(labels.dtype)

Found 75000 files belonging to 5 classes.
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'>
(64, 256, 256, 3)
<dtype: 'float32'>
(64,)
<dtype: 'int32'

In [6]:
from tensorflow.keras.layers import CenterCrop
from tensorflow.keras.layers import Rescaling

cropper = CenterCrop(height=30, width=30)
scaler = Rescaling(1.0/255)

dataset = scaler(cropper(dataset))

ValueError: Exception encountered when calling layer "center_crop_1" (type CenterCrop).

Attempt to convert a value (<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>) to a Tensor.

Call arguments received by layer "center_crop_1" (type CenterCrop):
  • inputs=<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
from tensorflow.keras import layers

inputs = keras.Input(shape=(256, 256, 3))

x = CenterCrop(height=30, width=30)(inputs)
x = Rescaling(scale=1.0/255)(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(1, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

data = np.random.randint(0, 256, size=(64, 256, 256, 3)).astype("float32")
processed_data = model(data)
print(processed_data)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3), loss=keras.losses.CategoricalCrossentropy())

In [ ]:
%load_ext tensorboard
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3), loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(dataset, epochs=2, callbacks=callbacks)

In [ ]:
import vitaldb
import matplotlib.pyplot as plt
vf = vitaldb.VitalFile(10, ['SNUADC/ECG_II', 'SNUADC/ART'])

samples = vf.to_numpy(['SNUADC/ECG_II', 'SNUADC/ART'], 1/500, False, True)
samples

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(samples[:,0], samples[:,1])
plt.show()

In [8]:
from vitaldbgenerator import VitalDBGenerator, VitalFileOptions
import matplotlib.pyplot as plt
import multiprocessing

def graphfile(file):
    plt.figure(figsize=(20,5))
    plt.plot(file[:,0], file[:,1])
    plt.show()

options = VitalFileOptions(
    ['SNUADC/ECG_II', 'SNUADC/ART'],
    1/500
)
vitalDbGenerator = VitalDBGenerator(options, 4)

pool = multiprocessing.Pool()
pool.map(graphfile, vitalDbGenerator)

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.10/3.10.4/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@3.10/3.10.4/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python@3.10/3.10.4/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/local/Cellar/python@3.10/3.10.4/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'graphfile' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.10/3.10.4/Frameworks/Python.framework/Version

KeyboardInterrupt: 

In [25]:
from vitaldbgenerator import VitalDBGenerator, VitalFileOptions
import tensorflow as tf

options = VitalFileOptions(
    ['SNUADC/ECG_II', 'SNUADC/ART'],
    1/500
)
vitalDbGenerator = VitalDBGenerator(options, 10)


dataset = tf.data.Dataset.from_generator(
    lambda: vitalDbGenerator,
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.float64)
    )
)

for data, labels in dataset:
    print(data.shape)
    print(data.dtype)
    print(labels.shape)
    print(labels.dtype)

2022-05-28 01:01:45.461733: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (5770913, 3) where an element of shape () was expected.
Traceback (most recent call last):

  File "/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1073, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (5770913, 3) where an element of shape () was expected.




InvalidArgumentError: TypeError: `generator` yielded an element of shape (5770913, 3) where an element of shape () was expected.
Traceback (most recent call last):

  File "/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1073, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (5770913, 3) where an element of shape () was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

In [30]:
from vitaldbgenerator import VitalDBGenerator, VitalFileOptions
import matplotlib.pyplot as plt

options = VitalFileOptions(
    ['SNUADC/ART', 'SNUADC/PLETH'],
    1/500
)
vitalDbGenerator = VitalDBGenerator(options, 4)

vitalDbGenerator.next()

array([[0.0000000e+00,           nan,           nan],
       [2.0000000e-03,           nan,           nan],
       [4.0000000e-03,           nan,           nan],
       ...,
       [1.1541820e+04,           nan,           nan],
       [1.1541822e+04,           nan,           nan],
       [1.1541824e+04,           nan,           nan]])